## IOT Readings - Embbed all readings or not?
Test dataset to evaluate RU cost for collocating IOT readings in a single document or to have multiple documents with N readings. </br>
Maximum number of readings: 10000

### Prepare key references for notebook
Expected CosmosDBAccounts.cfg with format:<br/>
[CosmosDB]<br/>
COSMOSDB_ACCOUNT_URI: https://CosmosDBAccount.documents.azure.com:443/<br/>
COSMOSDB_ACCOUNT_KEY: CosmosDBKey<br/>

In [1]:
from azure.cosmos import CosmosClient, PartitionKey
from configparser import ConfigParser
from faker import Faker

import os
import json
import uuid

parser = ConfigParser()
parser.read('../NotebookConfig.cfg')

cosmosAccountURI = parser.get('CosmosDB', 'COSMOSDB_ACCOUNT_URI')
cosmosAccountKey = parser.get('CosmosDB', 'COSMOSDB_ACCOUNT_KEY')

databaseName = 'Models'
containerName = 'Cnt5PhysicalPartitions'
partitionKeypath = '/PartitionKey'

Faker.seed(0)
fake = Faker(['en-US'])

print(cosmosAccountURI)

https://cosmicgbbcdb-sql.documents.azure.com:443/


### Create DB and collection for sample data
If resources already exists, just get references for database and container.

In [2]:
client = CosmosClient(cosmosAccountURI, cosmosAccountKey)
db = client.create_database_if_not_exists(databaseName)

pkPath = PartitionKey(path=partitionKeypath)
ctr = db.create_container_if_not_exists(id=containerName, partition_key=pkPath, offer_throughput=1000) 

### Repeatable Reference lists
Contains documents that should have consistent across cell/operations. </br>
For example: Patient, IOT devices, tax payer information, ...

In [3]:
from collections import OrderedDict
maxRange = 10000
IOTSources = []

os.makedirs(os.path.dirname('./OutputFiles/'), exist_ok=True)
with open('./OutputFiles/' + containerName + '_referenceData.json', 'w') as jsonFile:
    for i in range(maxRange):
        entity = {            
            'Name': fake.bothify('????_############') ## SiteId_ResourceId
            , 'Type': fake.random_element(elements=('Type1', 'Type2', 'Type3', 'Type4', 'Type5', 'Type6'))
        }
        IOTSources.append(entity)

        # Save patients for reference
        json.dump(entity, jsonFile)
        if (i < maxRange):
            jsonFile.write(',\n')

### Load sample documents in the container
Save generated documents in 'Output' directory

In [6]:
RUCharges = []
docs = []
maxRange = 1

for j in range(maxRange):
    IOTSrc = IOTSources[fake.random_int(min=0, max=9999)]    
    readings = []

    # *** Produce N readings
    # Execute this repeatedly by changing range from 1, 100, 1000 and 10000
    for r in range(1):        
        readings.append(
            {
                'Dimension': fake.random_element(elements=('D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'))
                , 'Value': fake.random_number(digits=5)
                , 'Timestamp': fake.date_time_this_year().isoformat()
            })

    doc = {
        'id': str(uuid.uuid4())
        , 'PartitionKey': fake.bothify('##_') + IOTSrc['Name'] + '_' + IOTSrc['Type']
        , 'Entity': IOTSrc
        , 'Readings': readings
        , 'class': fake.random_element(elements=OrderedDict([("A", 0.40), ("B", 0.35), ("C", 0.15), ("D", 0.05), ("E", 0.05)]))
    }

    # Create items, record charges and store docs locally (optional)
    # print(doc)
    ctr.create_item(doc)
    RUCharges.append(float(ctr.client_connection.last_response_headers['x-ms-request-charge']))
    # print(ctr.client_connection.last_response_headers['x-ms-request-charge'])
    docs.append(doc)

with open('./OutputFiles/' + containerName + '_1docs.json', 'w') as jf:
    for d in docs:
        json.dump(d, jf)
        jf.write(',\n')

In [7]:
print('Average RU charge: ' + str(sum(RUCharges) / len(RUCharges)))

Average RU charge: 9.33


In [7]:
RUCharges = []
docs = []
maxRange = 1

for j in range(maxRange):
    IOTSrc = IOTSources[fake.random_int(min=0, max=9999)]    
    readings = []

    # *** Produce N readings
    # Execute this repeatedly by changing range from 1, 100, 1000 and 10000
    for r in range(100):        
        readings.append(
            {
                'Dimension': fake.random_element(elements=('D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'))
                , 'Value': fake.random_number(digits=5)
                , 'Timestamp': fake.date_time_this_year().isoformat()
            })

    doc = {
        'id': str(uuid.uuid4())
        , 'PartitionKey': fake.bothify('##_') + IOTSrc['Name'] + '_' + IOTSrc['Type']
        , 'Entity': IOTSrc
        , 'Readings': readings
        , 'class': fake.random_element(elements=OrderedDict([("A", 0.40), ("B", 0.35), ("C", 0.15), ("D", 0.05), ("E", 0.05)]))
    }

    # Create items, record charges and store docs locally (optional)
    # print(doc)
    ctr.create_item(doc)
    RUCharges.append(float(ctr.client_connection.last_response_headers['x-ms-request-charge']))
    # print(ctr.client_connection.last_response_headers['x-ms-request-charge'])
    docs.append(doc)

with open('./OutputFiles/' + containerName + '_100docs.json', 'w') as jf:
    for d in docs:
        json.dump(d, jf)
        jf.write(',\n')

In [8]:
print('Average RU charge: ' + str(sum(RUCharges) / len(RUCharges)))

Average RU charge: 69.52


In [8]:
RUCharges = []
docs = []
maxRange = 1000

for j in range(maxRange):
    IOTSrc = IOTSources[fake.random_int(min=0, max=9999)]    
    readings = []

    # *** Produce N readings
    # Execute this repeatedly by changing range from 1, 100, 1000 and 10000
    for r in range(1000):        
        readings.append(
            {
                'Dimension': fake.random_element(elements=('D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'))
                , 'Value': fake.random_number(digits=5)
                , 'Timestamp': fake.date_time_this_year().isoformat()
            })

    doc = {
        'id': str(uuid.uuid4())
        , 'PartitionKey': fake.bothify('##_') + IOTSrc['Name'] + '_' + IOTSrc['Type']
        , 'Entity': IOTSrc
        , 'Readings': readings
        , 'class': fake.random_element(elements=OrderedDict([("A", 0.40), ("B", 0.35), ("C", 0.15), ("D", 0.05), ("E", 0.05)]))
    }

    # Create items, record charges and store docs locally (optional)
    # print(doc)
    ctr.create_item(doc)
    RUCharges.append(float(ctr.client_connection.last_response_headers['x-ms-request-charge']))
    # print(ctr.client_connection.last_response_headers['x-ms-request-charge'])
    docs.append(doc)

with open('./OutputFiles/' + containerName + '_1000docs.json', 'w') as jf:
    for d in docs:
        json.dump(d, jf)
        jf.write(',\n')

KeyboardInterrupt: 

In [10]:
print('Average RU charge: ' + str(sum(RUCharges) / len(RUCharges)))

Average RU charge: 624.0


In [7]:
RUCharges = []
docs = []
maxRange = 10

for j in range(maxRange):
    IOTSrc = IOTSources[fake.random_int(min=0, max=9999)]    
    readings = []

    # *** Produce N readings
    # Execute this repeatedly by changing range from 1, 100, 1000 and 10000
    for r in range(10000):        
        readings.append(
            {
                'Dimension': fake.random_element(elements=('D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'))
                , 'Value': fake.random_number(digits=5)
                , 'Timestamp': fake.date_time_this_year().isoformat()
            })

    doc = {
        'id': str(uuid.uuid4())
        , 'PartitionKey': fake.bothify('##_') + IOTSrc['Name'] + '_' + IOTSrc['Type']
        , 'Entity': IOTSrc
        , 'Readings': readings
        , 'class': fake.random_element(elements=OrderedDict([("A", 0.40), ("B", 0.35), ("C", 0.15), ("D", 0.05), ("E", 0.05)]))
    }

    # Create items, record charges and store docs locally (optional)
    # print(doc)
    ctr.create_item(doc)
    RUCharges.append(float(ctr.client_connection.last_response_headers['x-ms-request-charge']))
    # print(ctr.client_connection.last_response_headers['x-ms-request-charge'])
    docs.append(doc)

with open('./OutputFiles/' + containerName + '_10000docs.json', 'w') as jf:
    for d in docs:
        json.dump(d, jf)
        jf.write(',\n')

CosmosHttpResponseError: (429) Message: {"Errors":["Request rate is large. More Request Units may be needed, so no changes were made. Please retry this request later. Learn more: http://aka.ms/cosmosdb-error-429"]}
ActivityId: eddbb7e1-1287-41ff-b220-a1b381a55ce3, Request URI: /apps/38bfe303-da54-4990-99d7-2e697e386886/services/ded8d209-afd6-4758-a06d-9adabc3d0df8/partitions/300bc55c-bca2-46c6-9ed3-68059f435abe/replicas/133151112070723957p/, RequestStats: 
RequestStartTime: 2023-01-31T14:47:07.3963717Z, RequestEndTime: 2023-01-31T14:47:07.3963717Z,  Number of regions attempted:1
{"systemHistory":[{"dateUtc":"2023-01-31T14:46:11.6259969Z","cpu":0.195,"memory":440055952.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.0123,"availableThreads":32763,"minThreads":64,"maxThreads":32767},"numberOfOpenTcpConnection":1087},{"dateUtc":"2023-01-31T14:46:21.6360656Z","cpu":0.246,"memory":440037164.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.016

In [12]:
print('Average RU charge: ' + str(sum(RUCharges) / len(RUCharges)))

# Interesting observation, can I execute a 6250 RU operations to insert a document in a 1000 RU container?

Average RU charge: 627.29


### Adjust index policy
The readings being stored in the container should not require indexing, as most of the operations should fetch documents based on Device and then compute the values. Therefore application will not benefit from default indexing. <br/>
Execute cell below to adjust indexPolicy and then re-execute cell to load data (adjusting number of readings)

In [10]:
# containerPath = 'dbs/'+ databaseName +'/colls/' + containerName
# container = db.get_container_client(container=containerName)

indexPolicy = {
    "indexingMode":"consistent",
    "includedPaths":[
        {"path":"/PartitionKey/?"}
        , {"path":"/Entity/*"}
        , {"path":"/Class/?"}
        , {"path":"/_ts/?"}
        ]
    , "excludedPaths":[{"path":"/*"}]
}

db.replace_container(containerName, pkPath, indexing_policy=indexPolicy)


<ContainerProxy [dbs/Models/colls/IOTEnergyTelemetry]>

### Cross partition query workload assessment
- Target 1 million readings
  - 10000 docs with 100 readings
  - 1000 docs with 1000 readings
  - 100 docs with 10000 readings

In [22]:
RUCharges = []
docs = []
maxRange = 10000
IOTSrc = IOTSources[fake.random_int(min=0, max=9999)]

for j in range(maxRange):
    readings = []

    # *** Produce N readings
    for r in range(100):        
        readings.append(
            {
                'Dimension': fake.random_element(elements=('D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'))
                , 'Metric': fake.random_number(digits=5)
                , 'Timestamp': fake.date_time_this_year().isoformat()
            })

    doc = {
        'id': str(uuid.uuid4())
        # , 'PartitionKey': fake.bothify('##_') + IOTSrc['Name'] + '_' + IOTSrc['Type']
        , 'PartitionKey': IOTSrc['Name'] + '_' + IOTSrc['Type']
        , 'Entity': IOTSrc
        , 'Readings': readings
        , 'class': fake.random_element(elements=OrderedDict([("A", 0.40), ("B", 0.35), ("C", 0.15), ("D", 0.05), ("E", 0.05)]))
    }

    # Create items, record charges and store docs locally (optional)
    # print(doc)
    ctr.create_item(doc)
    RUCharges.append(float(ctr.client_connection.last_response_headers['x-ms-request-charge']))
    # print(ctr.client_connection.last_response_headers['x-ms-request-charge'])
    docs.append(doc)

with open('./OutputFiles/' + containerName + '_Opt1_docs.json', 'w') as jf:
    for d in docs:
        json.dump(d, jf)
        jf.write(',\n')

In [ ]:
RUCharges = []
docs = []
maxRange = 1000
IOTSrc = IOTSources[fake.random_int(min=0, max=9999)]

for j in range(maxRange):
    readings = []

    # *** Produce N readings
    for r in range(1000):        
        readings.append(
            {
                'Dimension': fake.random_element(elements=('D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'))
                , 'Metric': fake.random_number(digits=5)
                , 'Timestamp': fake.date_time_this_year().isoformat()
            })

    doc = {
        'id': str(uuid.uuid4())
        # , 'PartitionKey': fake.bothify('##_') + IOTSrc['Name'] + '_' + IOTSrc['Type']
        , 'PartitionKey': IOTSrc['Name'] + '_' + IOTSrc['Type']
        , 'Entity': IOTSrc
        , 'Readings': readings
        , 'class': fake.random_element(elements=OrderedDict([("A", 0.40), ("B", 0.35), ("C", 0.15), ("D", 0.05), ("E", 0.05)]))
    }

    # Create items, record charges and store docs locally (optional)
    # print(doc)
    ctr.create_item(doc)
    RUCharges.append(float(ctr.client_connection.last_response_headers['x-ms-request-charge']))
    # print(ctr.client_connection.last_response_headers['x-ms-request-charge'])
    docs.append(doc)

with open('./OutputFiles/' + containerName + '_Opt1_docs.json', 'w') as jf:
    for d in docs:
        json.dump(d, jf)
        jf.write(',\n')

In [ ]:
RUCharges = []
docs = []
maxRange = 100
IOTSrc = IOTSources[fake.random_int(min=0, max=9999)]

for j in range(maxRange):
    readings = []

    # *** Produce N readings
    for r in range(10000):        
        readings.append(
            {
                'Dimension': fake.random_element(elements=('D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'))
                , 'Metric': fake.random_number(digits=5)
                , 'Timestamp': fake.date_time_this_year().isoformat()
            })

    doc = {
        'id': str(uuid.uuid4())
        # , 'PartitionKey': fake.bothify('##_') + IOTSrc['Name'] + '_' + IOTSrc['Type']
        , 'PartitionKey': IOTSrc['Name'] + '_' + IOTSrc['Type']
        , 'Entity': IOTSrc
        , 'Readings': readings
        , 'class': fake.random_element(elements=OrderedDict([("A", 0.40), ("B", 0.35), ("C", 0.15), ("D", 0.05), ("E", 0.05)]))
    }

    # Create items, record charges and store docs locally (optional)
    # print(doc)
    ctr.create_item(doc)
    RUCharges.append(float(ctr.client_connection.last_response_headers['x-ms-request-charge']))
    # print(ctr.client_connection.last_response_headers['x-ms-request-charge'])
    docs.append(doc)

with open('./OutputFiles/' + containerName + '_Opt1_docs.json', 'w') as jf:
    for d in docs:
        json.dump(d, jf)
        jf.write(',\n')

### Clean up code

In [ ]:
# containerPath = 'dbs/'+ databaseName +'/colls/' + containerName
# container = db.get_container_client(container=containerName)

indexPolicy = {
    "indexingMode":"consistent",
    "includedPaths":[
        {"path":"/*"}
        ]
    , "excludedPaths":[{"path":"/\"_etag\"/?"}]
}

db.replace_container(containerName, pkPath, indexing_policy=indexPolicy)

In [2]:
# Assume objects are instantiated
# db.delete_container(containerName)

import glob

files = glob.glob('.\OutputFiles\*')
for f in files:
    os.remove(f)